In [1]:
!pip install deepface opencv-python matplotlib pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.2/169.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 2.9 MB/s eta 0:00:00


In [2]:
import cv2
import os
import numpy as np
import pandas as pd
import pickle
from deepface import DeepFace
from datetime import datetime

26-01-29 08:05:44 - Directory /root/.deepface has been created
26-01-29 08:05:44 - Directory /root/.deepface/weights has been created


In [3]:
os.makedirs("registered_faces", exist_ok=True)
os.makedirs("database", exist_ok=True)
os.makedirs("logs", exist_ok=True)

In [5]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def capture_image(filename):
    js = Javascript('''
    async function takePhoto() {
        const div = document.createElement('div');
        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});
        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        await new Promise(resolve => setTimeout(resolve, 3000));

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getTracks()[0].stop();
        div.remove();

        return canvas.toDataURL('image/jpeg', 0.8);
    }
    takePhoto();
    ''')
    display(js)
    data = eval_js("takePhoto()")
    img = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(img)


In [6]:
def register_user(name):
    path = f"registered_faces/{name}.jpg"
    capture_image(path)

    embedding = DeepFace.represent(
        img_path=path,
        model_name="Facenet",
        detector_backend="retinaface"
    )[0]["embedding"]

    db_path = "database/embeddings.pkl"

    if os.path.exists(db_path):
        with open(db_path, "rb") as f:
            db = pickle.load(f)
    else:
        db = {}

    db[name] = embedding

    with open(db_path, "wb") as f:
        pickle.dump(db, f)

    print(f"{name} registered successfully!")


In [8]:
register_user("meghana")


<IPython.core.display.Javascript object>

Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet_weights.h5
To: /root/.deepface/weights/facenet_weights.h5


26-01-29 08:07:05 - 🔗 facenet_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/facenet_weights.h5 to /root/.deepface/weights/facenet_weights.h5...


100%|██████████| 92.2M/92.2M [00:00<00:00, 240MB/s]
Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5


26-01-29 08:07:10 - retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


100%|██████████| 119M/119M [00:00<00:00, 204MB/s]


meghana registered successfully!


In [9]:
from numpy.linalg import norm

def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))


In [10]:
def identify_user(img_path):
    with open("database/embeddings.pkl", "rb") as f:
        db = pickle.load(f)

    test_emb = DeepFace.represent(
        img_path=img_path,
        model_name="Facenet",
        detector_backend="retinaface"
    )[0]["embedding"]

    best_match = None
    highest_score = 0

    for name, emb in db.items():
        score = cosine_similarity(test_emb, emb)
        if score > highest_score:
            highest_score = score
            best_match = name

    if highest_score > 0.6:
        return best_match, highest_score
    else:
        return None, highest_score


In [11]:
def spoof_check(img_path):
    try:
        DeepFace.detectFace(img_path, detector_backend="retinaface")
        return True
    except:
        return False


In [12]:
def mark_attendance(name, action):
    file = "logs/attendance.csv"

    time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    row = pd.DataFrame([[name, action, time]],
                       columns=["Name","Action","Time"])

    if os.path.exists(file):
        row.to_csv(file, mode="a", header=False, index=False)
    else:
        row.to_csv(file, index=False)

    print(f"{name} {action} at {time}")


In [13]:
def authenticate(action):
    capture_image("temp.jpg")

    if not spoof_check("temp.jpg"):
        print("Spoof Attempt Detected")
        return

    name, score = identify_user("temp.jpg")

    if name:
        print(f"Recognized: {name} ({round(score,2)})")
        mark_attendance(name, action)
    else:
        print("Unknown Person")


In [14]:
authenticate("Punch-In")
authenticate("Punch-Out")


<IPython.core.display.Javascript object>

26-01-29 08:08:45 - ⚠️ Function detectFace is deprecated. Use extract_faces instead.
Recognized: meghana (1.0)
meghana Punch-In at 2026-01-29 08:09:01


<IPython.core.display.Javascript object>

26-01-29 08:09:05 - ⚠️ Function detectFace is deprecated. Use extract_faces instead.
Recognized: meghana (0.99)
meghana Punch-Out at 2026-01-29 08:09:20


In [15]:
!find /content -type d


/content
/content/.config
/content/.config/configurations
/content/.config/logs
/content/.config/logs/2026.01.16
/content/registered_faces
/content/database
/content/logs
/content/sample_data


In [16]:
import os
import glob

files = glob.glob("/content/*.jpg") + glob.glob("/content/*.png")

for f in files:
    os.remove(f)

print("All saved images deleted ✅")


All saved images deleted ✅
